In [ ]:
import numpy as np
import pandas as pd
from pandas.plotting import autocorrelation_plot
import matplotlib.pyplot as plt
import seaborn as sns
from fbprophet import Prophet
from statsmodels.tsa.stattools import adfuller
import scipy

df = pd.read_csv(r'temperature.csv', parse_dates=[0], header=[0], index_col=[0])
df.columns = ['Temperature (°C)']
df['abs'] = range(len(df))
df.index = df.index + 3*(pd.to_timedelta(99999999999990000)-pd.to_timedelta('09:46:39.999990'))*3
df['linear'] = 0.0013246575342465753*df['abs']-0.0100212592764251
df['sine'] = 6*np.sin(df['abs']*np.pi/(182.5)+14)+13
df['added'] = df['linear'] + df['sine']
df['noise'] = 0.6*(df["Temperature (°C)"]-df['linear']-df['sine'])
df['ds'] = df.index
df['y'] = df['Temperature (°C)']
df['homoscedasticity'] = df.iloc[:,0].rolling(window=30).std()

# unfortunately the averaging is some work -.-
def averaging(row):
  try: 
    # average over the last four years:
    # Also, why on earth would you exclude the month/year timedelta???
    # If you are so bothered by it, do pay attention to the 30.5 days the average month has
    # and the 365.25days the average year has oO!
    avg = np.mean([df.loc[row.name-pd.Timedelta(x*365, unit="d")][0] for x in range(5)])
  except KeyError as e:
    # NAN if 4 years haven't passed yet
    return np.NaN
  return avg
df['averaging'] =  df.apply(averaging, axis=1)
sns.set(style="darkgrid")

In [ ]:
# To obtain the noise description - a normal distribution fit:
_=plt.hist(df['noise'], bins=25, density=True, alpha=0.6, color='g')
from scipy.stats import norm
mu, std = norm.fit(df['noise'])
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
_=plt.plot(x, p, 'k', linewidth=2)
_=plt.title(f"Fit results: mu = {mu:.2f},  std = {std:.2f}")
# 2 * std = [5,95] confidence interval:
lower_bound = df['added'] - 2*std
upper_bound = df['added'] + 2*std

In [ ]:
sns.set(style="darkgrid")
plt.plot(figsize=(6,6))
sns.lineplot(x=df.index, y=df['Temperature (°C)'], data=df, label='Real Data')
# sns.lineplot(x=df.index, y=df['averaging'], data=df, label='Averaging')
# sns.lineplot(x=df.index, y=df['homoscedasticity'], data=df, label='30d Variance')
# sns.lineplot(x=df.index, y=df['linear'], data=df, label='Linear Trend')
# sns.lineplot(x=df.index, y=df['sine'], data=df, label='Periodic Part')
sns.lineplot(x=df.index, y=df['added'], data=df, label='Model')
plt.fill_between(df.index, lower_bound, upper_bound, alpha=.3, label='std', color="orange")
plt.ylim(-1,30.5)
plt.ylabel('Temperature (°C)')
plt.tight_layout()

In [ ]:
# Showing that the first discrete difference can get rid of all seasonality&trend:
sns.lineplot(x=df.index, y=df['Temperature (°C)'], data=df, label='Real Data')
sns.lineplot(x=df.index, y=df['Temperature (°C)'].diff(1), data=df, label='Discrete Difference')
plt.ylim(-10,30.5)
plt.ylabel('Temperature (°C)')
plt.tight_layout()

In [ ]:
# Variance (30d Variance) and mean (noise) plot to show homoscedasticity:
sns.set(style="darkgrid")
plt.plot(figsize=(6,6))
sns.lineplot(x=df.index, y=df['noise'], data=df, label='Residual Noise')
sns.lineplot(x=df.index, y=df['sine'], data=df, label='Seasonality Fit')
sns.lineplot(x=df.index, y=df['homoscedasticity'], data=df, label='30d Variance')
plt.ylabel('Temperature (°C)')
plt.legend(loc='upper right')
plt.ylim(-7,24.5)
plt.tight_layout()

In [ ]:
# Getting started on the facebook Prophet modelling:
m = Prophet()
m.fit(df)

In [ ]:
# Creating the necessary in;ut cols for prophet's modelling:
future = m.make_future_dataframe(periods=365)
future.tail()
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
fig1 = m.plot(forecast)
sns.set(style="darkgrid")
plt.plot(figsize=(6,6))
plt.ylabel('Temperature (°C)')
plt.legend(['Data','Fit & CI'],loc='upper right')
plt.ylim(-1,30.5)

In [ ]:
# component plot:
sns.set(style="darkgrid")
fig2 = m.plot_components(forecast)
plt.tight_layout()

# multi var modelling hereafter

In [ ]:
# let's look at the australian stonks market:
stonks = pd.read_csv(r'AXJO.csv')
stonks['Date'] = pd.to_datetime(stonks['Date'])
stonks.index = stonks['Date']
stonks['ds'] = stonks['Date']
stonks['y'] = stonks['Open']

In [ ]:
plt.plot(figsize=(6,6))
sns.set(style="darkgrid")
ax1 = stonks['Open'].plot(label='ASX200 Opening Price')
ax2 = (stonks['Volume']/1000).plot(label='Volume', secondary_y=True)
ax1.set_ylabel('ASX200 Open Price in A$')
ax1.yaxis.label.set_color('blue')
ax2.set_ylabel('Trade Volume (1k)')
ax2.yaxis.label.set_color('orange')
# ax2.yaxis.label.set_scolor('orange')
plt.tight_layout()

In [ ]:
m = Prophet()
m.fit(stonks[stonks['Date']<'2020-02-10'])

In [ ]:
future = m.make_future_dataframe(periods=365)
future.tail()
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

sns.set(style="darkgrid")
m.plot(forecast)
plt.ylabel('ASX200 Opening Prices(A$)')
plt.legend(['Data','Fit & CI'],loc='top left')

In [ ]:
sns.set(style="darkgrid")
fig2 = m.plot_components(forecast)
plt.tight_layout()

In [ ]:
# The little brother of Cross-correlation: autocorrelation_plot!
# autocorrelation_plot is really helpful to determin the best values to build discrete differences
# if, e.g., autocorrelation_plot(df['noise']) shows a massive dependence on lag=15, then one should use it
# as a discrete difference step along the lines of df['noise'].diff(15)
autocorrelation_plot(df['noise'], label='acf of noise')
_ = plt.xlabel('Lag (days)')
_ = plt.ylim((-.3,.3))
# plt.specgram(df['y'].dropna(),fs=185)

In [ ]:
# Same plot for 1st 10 days
plt.acorr(df['noise'], label='Autocorrelations <10d')#
plt.legend()
plt.xlim(-0.05,10.05)
# df['noise'].autocorr(80)

In [ ]:
# Same for the "stonks" ;)
autocorrelation_plot(stonks['Open'], label='Open')
autocorrelation_plot(stonks['Volume'], label='Volume')

### Some of the analysis tools:

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mutual_info_score

# loading data:

df_all = pd.read_csv('./data/df_all.csv', header=[0], index_col=[0],parse_dates=[0])            
df_all = df_all.sort_index()
df_all.index = pd.to_datetime(df_all.index)
# silly gluonTS needs an index with freq set:
df_all.index = pd.date_range(start=df_all.index[0], freq='120min', periods=df_all.shape[0])


y = pd.read_csv('./data/y.csv', header=[0], index_col=[0],parse_dates=[0])          
y = y.sort_index()
y.index = pd.to_datetime(y.index)
y.index = pd.date_range(start=y.index[0], freq='120min', periods=y.shape[0])

# reality check:
assert all(dat in df_all.index for dat in y.index), 'Index differs for df_all and y!'

print('df_all.shape: ',df_all.shape)
print('y.shape: ',y.shape)

In [ ]:
sns.set(style="darkgrid")

temp = df_all[['temperature_SA','windSpeed_SA','cloudCover_SA','scheduledGeneration_SA']
              ].merge(y['price_SA'], 
                      left_index=True, 
                      right_index=True).copy()

cor = temp.corr()

# plot the heatmap:
plt.figure(figsize=(6,4))
_ = sns.heatmap(cor,
                xticklabels=cor.columns,
                yticklabels=cor.columns,
                annot=True,
                fmt='.1f')
_ = plt.xticks(rotation=90)
_ = plt.yticks(rotation=45)
del temp
# plt.tight_layout()

In [ ]:
# Right, let's get started with the (cross-) correlations:
def crosscorr(df_x, df_y, lags=[0]):
    # calculates cross correlation
    cross_corr = [df_x.corr(df_y.shift(lag)) for lag in lags]
    return cross_corr

In [ ]:
temp = df_all[['pressure_SA','temperature_SA',               
               'rainProb_SA','windSpeed_SA',
               'windDirection_SA','cloudCover_SA',
               'scheduledGeneration_SA']].merge(y['price_SA'],
                                                left_index=True,
                                                right_index=True).copy()

abc = {col: crosscorr(temp[col],temp['price_SA'],range(50)) for col in temp.columns if col!='price_SA'}
abc = pd.DataFrame(abc)
sns.set(style="darkgrid")
abc.index.name = 'Lag (h)'

sns.heatmap(abc.T, robust=True)#, cmap=cmap)
plt.plot(figsize=(6,6))

In [ ]:
# Do be careful with correlations please, they are not the catch-all that will bring you world peace!
a = np.arange(-15,15)
b = np.arange(-15,15)**2
c = np.arange(-25,25)
d = np.arange(-25,25)**2

ab = pd.DataFrame({'x':a,'y':b})
cd = pd.DataFrame({'x':c,'y':d})
corr1 = ab.corr()
corr2 = cd.corr()

_=sns.set(style="darkgrid")
_=plt.plot(c,d,label='[-25,25]')
_=plt.plot(a,b,label='[-15,15]')
_=plt.legend(loc='best')
_=plt.title(f'Correlation [-15,15]: {corr1.iloc[0,1]:.2f} - Correlation [-25,25]: {corr2.iloc[0,1]:.2f}')
_=plt.plot(figsize=(6,6))

# Right, now for the modelling!

In [ ]:
# Let's kick off with gluonTS: (caveat: may be a super nice package, defo not as performant yet in pure regression in comp to e.g. tf/torch)
#
# Do change the epoch number to see slightly better results
#
from gluonts.dataset import common
from gluonts.dataset.field_names import FieldName
from gluonts.dataset.common import ListDataset
from gluonts.model import deepar, wavenet
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
import mxnet as mx
from mxnet import gluon
import os
from tqdm.autonotebook import tqdm
from pathlib import Path

prediction_length = 24
start = pd.Timestamp('2019-11-11 03:00:00') # df_all.index[0]
freq = '120min'

df_train = df_all[['scheduledGeneration_SA','temperature_SA', 'rainProb_SA',
       'windSpeed_SA', 'cloudCover_SA']]

train_ds = ListDataset([{FieldName.TARGET: 
                                    y.iloc[:-prediction_length, 2].values, 
                         FieldName.FEAT_DYNAMIC_REAL: 
                                    df_train.iloc[:-prediction_length,:].values,
                         FieldName.START: 
                                    start}],
                       freq=freq)
# test dataset: use the whole dataset, add "target" and "start" fields
test_ds = ListDataset([{FieldName.TARGET: 
                                    y.iloc[:, 2].values, 
                        FieldName.FEAT_DYNAMIC_REAL: 
                                    df_train.values,
                        FieldName.START: 
                                    start}],
                       freq=freq)


trainer = Trainer(epochs=10)
estimator = deepar.DeepAREstimator(
    freq="120min", prediction_length=prediction_length, trainer=trainer)
predictor = estimator.train(training_data=train_ds)

prediction = next(predictor.predict(test_ds))
# print(prediction.mean)
# prediction.plot(output_file='graph.png')

forecast_it, ts_it = make_evaluation_predictions(
    dataset=test_ds,  # test dataset
    predictor=predictor,  # predictor
    num_samples=100,  # number of sample paths we want for evaluation
)
forecasts = list(forecast_it)
tss = list(ts_it)

In [ ]:
# As one can tell, the model still leaves room for improvement:
def plot_prob_forecasts(ts_entry, forecast_entry):
    # plot everything
    sns.set(style="darkgrid")

    plot_length = 50
    prediction_intervals = (50.0, 90.0)
    legend = ["observations", "median prediction"] + [f"{k}% prediction interval" for k in prediction_intervals][::-1]

    fig, ax = plt.subplots(1, 1, figsize=(10, 7))
    ts_entry[-plot_length:].plot(ax=ax)  # plot the time series
    forecast_entry.plot(prediction_intervals=prediction_intervals, color='g')
    plt.grid(which="both")
    plt.legend(legend, loc="upper left")
    
plot_prob_forecasts(tss[0], forecasts[0])

### WaveNet:

In [ ]:
# Pick and mix your input variables (`df_train`) to tune model performance!

from tensorflow.keras.layers import Dense
from tensorflow.keras import Input, Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from tcn import TCN, tcn_full_summary

df_train = df_all[['temperature_SA','cloudCover_SA','windSpeed_SA','scheduledGeneration_SA']]
batch_size, timesteps, input_dim = None, df_train.shape[1], 1

i = Input(batch_shape=(batch_size, timesteps, input_dim))
o = TCN(kernel_size=2,
        dilations=[1, 2, 4, 8, 16, 32],
        return_sequences=True)(i)
o = TCN(kernel_size=2,
        dilations=[1, 2, 4, 8, 16, 32],
        return_sequences=False)(o)
o = Dense(1)(o)

m3 = Model(inputs=[i], outputs=[o])

m3.compile(optimizer='adam', loss='mse')


x_train = np.array(df_train.iloc[:-25,:].values.reshape(-1,df_train.shape[1],1))
y_train = np.array(y.iloc[:-25,2].values.reshape(-1,1))


name_of_run = '1'
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=150)
mc = ModelCheckpoint(f'./models/run_{name_of_run}/', monitor='val_loss',
                     verbose=0, save_best_only=True,
                     save_weights_only=False, mode='auto')

m3.fit(x_train, y_train, epochs=2000, validation_split=0.15, callbacks=[es, mc])

In [ ]:
# m3.save('./models/model_WaveNet_500epochs-2layer-extend-ES/')

In [ ]:
# plot val-/ train- loss
plt.plot(m3.history.history['loss'], label='train')
plt.plot(m3.history.history['val_loss'], label='test')
plt.legend(loc='best')
plt.yscale('log')

In [ ]:
# Plotting the fit results together with real data:

import tensorflow 

# m3 = tensorflow.keras.models.load_model('./models/model_WaveNet_500epochs-2layer-extend-ES/')

m4 = tensorflow.keras.models.load_model(f'./models/run_{name_of_run}')

test_series_x = np.array(df_train.iloc[-126:-100,:].values.reshape(-1,df_train.shape[1],1))

preds_m3 = m3.predict(test_series_x)
preds_m4 = m4.predict(test_series_x)

In [ ]:
test_series_y = y.iloc[-175:-125,2]
train_series_y = y.iloc[-126:-100,2]

pred_y = pd.DataFrame({"Model_0": preds_m3.ravel(),
                  "Model_1 - best of 0": preds_m4.ravel()},
                  index=train_series_y.index
                  )

fig, ax = plt.subplots(1, figsize=(10, 7))
train_series_y.plot(ax=ax)
plt.axvline(test_series_y.index[-1], color='black')

test_series_y.plot(ax=ax)

test_series_x = np.array(df_all.iloc[-126:-100,:].values.reshape(26,21,1))
idx = df_all.iloc[-126:-100,:].index
idx.freq=None

# pred_y['Model_0'].plot(ax=ax)
pred_y['Model_1 - best of 0'].plot(ax=ax)
plt.ylabel('AUD/MWh')
plt.legend(["Test series", "End of train series", 'Train series','WaveNet: Best of 500 Epochs'])